<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

***

# 3.0 ROS Integration of Object Detection

In this third and final section, you will train an object detection model with NVIDIA DIGITS.  Similar to the image recognition section, you will integrate a pre-trained deployed model into ROS using the `detectnet` ROS node from the `ros_deep_learning` package. You'll test your code in simulation and then deploy it to the physical robot. 

[3.1 What is Object Detection?](#object_detection)<br>
[3.2 Object Detection Data](#data)<br>
[3.3 Object Detection Models](#models)<br>
[3.4 Inference on the Robot's Jetson](#inference)<br>
[3.5 Integrating Inference into ROS](0350_IntegratingObjectDetectionROS.ipynb#integration)<br>
[3.6 Your ROS Object Detection node: bot-nav detector](0350_IntegratingObjectDetectionROS.ipynb#simulation)<br>
[3.7 Assessment](0350_IntegratingObjectDetectionROS.ipynb#assessment)<br>
[3.8 Deployment](0350_IntegratingObjectDetectionROS.ipynb#deployment)<br>
[3.9 Recap](0350_IntegratingObjectDetectionROS.ipynb#recap)<br>
[Appendix](9900_DL4R_Appendix.ipynb#apx-3)

<a id='object_detection'></a>
# 3.1 What is Object Detection?

**Object Detection** classifies objects, but goes a step further in that it localizes them within the image.  This way, multiple objects can be detected simultaneously.  Object detection is one of the most challenging problems in computer vision and is the first step in several computer vision applications.<br>
The goal of an object detection system is to detect all instances of objects of a known category in an image.

<img src="images/trucks_bb.png" alt="object_detection" width=600/>
The training workflow for object detection with DIGITS is the same as the training for an image classification network model.  

1. Create a dataset for a certain object detection task
- Define an object detection network model
- Train the model with the dataset

In this lab, we won't be able to actually run the training of the object detection model due to time constraints.<br>
However, we have some pre-created datasets and models, so we can see look at some examples of object detection training.

<a id='data'></a>
# 3.2 Object Detection Data
Object Detection data includes more than just object labels.  It includes localization information, generally in the form of bounding boxes.


### Public Datasets for Object Detection

There are some publicly available datasets that contain data for object detection.

- [MS COCO](http://cocodataset.org/#home)
- [Pascal VOC](http://host.robots.ox.ac.uk/pascal/VOC/)
- [KITTI](http://www.cvlibs.net/datasets/kitti/)
- [Open Images](https://github.com/openimages/dataset)

The datasets we will look at are derived from the COCO (Common Objects in Context) data. By going to the website, you can browse the various datasets and see visualizations.  

<img src="images/coco1.png" alt="coco" width=600/>
<img src="images/coco2.png" alt="coco" width=600/>

Although the browsing tool for COCO provides a robust visualization with many vertices, the Object Detection network we will use in DIGITS uses bounding box information. This information is also provided in the COCO data, but needs to be reformatted to the KITTI format for use with DIGITS.

### KITTI data format

For more detail of the KITTI data format, please check the README on Object Detection in the NVIDIA DIGITS repo [here](https://github.com/NVIDIA/DIGITS/blob/digits-6.0/digits/extensions/data/objectDetection/README.md).

The key information in the KITTI text file is 4 `bbox` fields which contain the information for a 2D bounding box of the object in the image: values are the left, top, right, and bottom pixel coordinates.  Note that the (0,0) location of an image is the upper left corner.

<img src="images/kittiformat.png" alt="kitti" width=800/>

### Conversion
Information on downloading and converting additional COCO data subsets into KITTI format can be found [here](https://github.com/dusty-nv/jetson-inference#downloading-the-detection-dataset) in the `jetson-inference` repo. 
A script, [coco2kitti.py](https://github.com/dusty-nv/jetson-inference/blob/master/tools/coco2kitti.py), is also available in the repo.

Explore converted COCO datasets provided below, ready for DIGITS: airplane, bottle, chair, and dog.  Execute the following cells to view the locations and examples.  Feel free to change the cells so you can look at different data subsets, images, and corresponding bounding box label text files. 

In [ ]:
!tree -d /dli/data/coco

In [ ]:
!ls /dli/data/coco/train/images/airplane

In [ ]:
from IPython.display import Image
Image("/dli/data/coco/train/images/airplane/110203.jpg")

In [ ]:
!cat /dli/data/coco/train/labels/airplane/110203.txt

### Your Dataset

### <a href="/digits/" target="_blank">OPEN DIGITS</a>
Click the link above to launch DIGITS if necessary.<br>
Keep both browser windows open (this notebook and DIGITS) so that you can refer to these instructions as you use the DIGITS tool. 

<img src="images/digits_cocosets.png" alt="digits_datasets" width=800/>

Click `Clone Job` to see how a KITTI dataset is constructed.

<img src="images/od_construction.png" alt="od_construction" width=800/>

To create a new dataset using the `bottle` data, complete the form using the following values and click "create":<br>

|Field|Path|
|-----|-----|
|Feature image folder |`/dli/data/coco/train/images/bottle`|
|Label image folder |`/dli/data/coco/train/labels/bottle`|
|Validation feature image folder |`/dli/data/coco/val/images/bottle`|
|Validation label image folder |`/dli/data/coco/val/labels/bottle`|
|Custom classses|`dontcare, bottle`|
|Dataset Name|`COCO-bottle`|


<a id='models'></a>
# 3.3 Object Detection models

Open DIGITS, and click on the "Model" tab.<br>
There are 7 jobs, which are all object detection model training runs done on subsets of the COCO dataset.

When you build jobs yourself, be sure to give unique and identifiable names to each job, showing some parameters and results of the training.

<img src="images/digits_od_models.png" alt="od_construction" width=800/>

Open the first model in the list.<br>
On the Loss-Accuracy graph, select the Epoch #3 to #100 to have better visibility of the curve. The very beginning includes a spike that overshadows later values.  To do this, move your cursor over the 0-100 mini-graph directly under the Loss-Accuracy graph.  The grayed area will appear showing the range for the main graph.
<img src="images/select_3to100.png" width=600/>
<img src="images/od_detectnet.png" alt="od_detectnet" width=800/>

To see how this training job was configured, click on "Clone" job.

You may notice that there is a list of 3-4 training jobs for the same combination of network and the data class.
As suggested by the last part of the name, they are trained with slightly different parameters.

Check how those changes affected the model training result. In practice, experimentation is necessary to find a good combination of parameters for your dataset and model.
<img src="images/iteration.png" alt="iteration" width=400/>

To create your own models, refer to the instructions titled [Creating DetectNet Model with DIGITS](https://github.com/dusty-nv/jetson-inference/blob/master/docs/detectnet-training.md#creating-detectnet-model-with-digits) found in the `jetson-inference` repo.

<a id='inference'></a>
# 3.4 Inference on the Robot's Jetson

Just as with image recognition models, you can use DIGITS features to test the trained model with static images or a camera stream. Once you build a model with acceptable accuracy, you can deploy it to the Jetson.  For this project, a model that detects bottles trained from the COCO bottle subset ***has already been deployed to the Jetson on the R1 robot.*** 

For information on how to deploy the model to the Jetson, please see the [Appendix](9900_DL4R_Appendix.ipynb#apx-3)

<a id='integration'></a>
## [--> Next: 3.5 Integrating Object Detection into ROS](0350_IntegratingObjectDetectionROS.ipynb)

<div style="border:2px solid black; background-color:#f2f2f2; padding:10px; width:400px; margin:auto;">
<b><a href="DLI Welcome Notebook.ipynb">Overview</a></b><br>
<b><a href="0100_DLforRoboticsControl.ipynb">1.0 Introduction to ROS Control</a></b><br>
<b><a href="0200_DLforRoboticsImageRecognition.ipynb">2.0 ROS Integration of Image Recognition</a></b><br>
<b><a href="0300_DLforRoboticsObjectDetection.ipynb">3.0 ROS Integration of Object Detection</a></b><br>
<b><a href="9900_DL4R_Appendix.ipynb">Appendix</a></b><br>
</div>

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>